In [1]:
!pip install wandb onnx onnxruntime -q
!sudo apt-get install texlive-latex-recommended 
!sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended  
!wget http://mirrors.ctan.org/macros/latex/contrib/type1cm.zip 
!unzip type1cm.zip -d /tmp/type1cm 
!cd /tmp/type1cm/type1cm/ && sudo latex type1cm.ins
!sudo mkdir /usr/share/texmf/tex/latex/type1cm 
!sudo cp /tmp/type1cm/type1cm/type1cm.sty /usr/share/texmf/tex/latex/type1cm 
!sudo texhash 
!apt install cm-super

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lmodern fonts-noto-mono fonts-urw-base35
  libfontenc1 libgs9 libgs9-common libharfbuzz-icu0 libidn11 libijs-0.35
  libjbig2dec0 li

In [2]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
import os
import time
import copy
import random
import onnx
from sklearn import metrics
from sklearn.metrics import roc_curve, RocCurveDisplay, auc
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
random.seed(42)
np.random.seed(42)
torch.cuda.manual_seed_all(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def set_size(width, fraction=1, subplots=(1, 1)):
    # Width of figure (in pts)
    fig_width_pt = width * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27
    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2
    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

In [6]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'

In [7]:
def build_dataset(batch_size):    
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    data_dir = '/content/drive/MyDrive/plantsdata'
    dataset = datasets.ImageFolder(os.path.join(data_dir))
    dataset.transform = data_transforms['test']

    # 90/10 split
    train_dataset, test_dataset = random_split(dataset, [0.9, 0.1])

    train_dataset.dataset.transform = data_transforms['train']
    test_dataset.dataset.transform = data_transforms['test']

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                               shuffle=True, num_workers=4)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=4)

    dataloaders = {'train': train_loader, 'test': test_loader}
    dataset_size = len(dataset)
    dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}
    class_names = dataset.classes

    return dataset

def build_network():
    network = resnet50(weights=ResNet50_Weights.DEFAULT)
    num_ftrs = network.fc.in_features

    # Add linear layer with number of classes
    network.fc = nn.Linear(num_ftrs, 2)

    return network.to(device)

def build_optimizer(network, optimizer, learning_rate):
    optimizer = optim.SGD(network.parameters(),
                          lr=learning_rate, momentum=0.9)
    return optimizer

def train_epoch(network, loader, optimizer, criterion, scheduler, dataset_sizes):
    network.train()
    confusion = torch.empty([0, 1])
    confusion = confusion.to(device)
    running_loss = 0.0
    running_corrects = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        #loss = F.nll_loss(network(data), target)
        with torch.set_grad_enabled(True):
            outputs = network(data)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)
        
        #cumu_loss += loss.item()
        
        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

        confusion = torch.cat((confusion, preds[:, None] / target.data[:, None]))

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({'train/batch_loss': loss.item()})

    scheduler.step()

    tp = torch.sum(confusion == 1).item()
    fp = torch.sum(confusion == float('inf')).item()
    tn = torch.sum(torch.isnan(confusion)).item()
    fn = torch.sum(confusion == 0).item()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f = 2 * ((precision * recall) / (precision + recall))
    
    epoch_loss = running_loss / dataset_sizes['train']
    epoch_acc = running_corrects.double() / dataset_sizes['train']
    
    return (epoch_loss, epoch_acc, precision, recall, f, tp, fp, tn, fn)

def test(network, loader, optimizer, criterion, dataset_sizes):
    network.eval()
    confusion = torch.empty([0, 1])
    confusion = confusion.to(device)
    probabilities = torch.empty([0])
    probabilities = probabilities.to(device)
    targets = torch.empty([0])
    targets = targets.to(device)
    running_loss = 0.0
    test_corrects = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        with torch.set_grad_enabled(False):
            outputs = network(data)
            _, preds = torch.max(outputs, 1)
            # Convert logits to probabilities
            targets = torch.cat((targets, target))
            probabilities = torch.cat((probabilities, F.softmax(outputs, dim=1)[:, 1]))
            loss = criterion(outputs, target)

        running_loss += loss.item() * data.size(0)
        test_corrects += torch.sum(preds == target.data)
        
        confusion = torch.cat((confusion, preds[:, None] / target.data[:, None]))

    tp = torch.sum(confusion == 1).item()
    fp = torch.sum(confusion == float('inf')).item()
    tn = torch.sum(torch.isnan(confusion)).item()
    fn = torch.sum(confusion == 0).item()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f = 2 * ((precision * recall) / (precision + recall))
    
    epoch_loss = running_loss / dataset_sizes['test']
    epoch_acc = test_corrects.double() / dataset_sizes['test']
    
    y_true = targets.detach().cpu().numpy()
    y_score = probabilities.detach().cpu().numpy()

    nn_fpr, nn_tpr, nn_thresholds = roc_curve(y_true, y_score)

    auc = metrics.roc_auc_score(y_true, y_score)
    
    return (epoch_loss, epoch_acc, precision, recall, f, tp, fp, tn, fn, y_true, y_score, auc)

In [8]:
def train(config=None):
    # Style the plots (with grid this time)
    width = 418
    sns.set_theme(style='whitegrid',
                  rc={'text.usetex': True, 'font.family': 'serif', 'axes.labelsize': 10,
                      'font.size': 10, 'legend.fontsize': 8,
                      'xtick.labelsize': 8, 'ytick.labelsize': 8})

    fig_save_dir = '../../thesis/graphics/'
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        dataset = build_dataset(config.batch_size)

        print("Dataset targets: ", dataset.targets)
        
        splits = StratifiedKFold(n_splits=config.k_splits, shuffle=True, random_state=42)
        foldperf={}

        # Aggregate metrics from best epochs for ROC plot
        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)

        # Style the plots (with grid this time)
        sns.set_palette('ch:light=0.8,gamma=1.2', n_colors=12)
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        
        # Iterate over the folds
        for fold, (train_idx, val_idx) in enumerate(splits.split(np.zeros(len(dataset)), dataset.targets)):

            print('Fold {}'.format(fold + 1))

            train_sampler = SubsetRandomSampler(train_idx)
            test_sampler = SubsetRandomSampler(val_idx)
            train_loader = DataLoader(dataset, batch_size=config.batch_size, sampler=train_sampler)
            test_loader = DataLoader(dataset, batch_size=config.batch_size, sampler=test_sampler)
            
            dataset_sizes = {'train': len(train_loader.sampler), 'test': len(test_loader.sampler)}

            print("Dataset sizes: ", dataset_sizes)

            network = build_network()
            optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
            criterion = nn.CrossEntropyLoss()
            # Decay LR by a factor of 0.1 every 5 epochs
            exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, config.step_size)

            best_test_auc = 0.0
            best_y_true = []
            best_y_score = []

            for epoch in tqdm(range(config.epochs)):            
                (train_loss, train_acc, train_precision, train_recall, train_f,
                 train_tp, train_fp, train_tn, train_fn) = train_epoch(network, train_loader, optimizer,
                                                                       criterion, exp_lr_scheduler,
                                                                       dataset_sizes)
                wandb.log({'epoch': epoch, 'train/epoch_loss': train_loss, 'train/epoch_acc': train_acc, 
                           'train/precision': train_precision, 'train/recall': train_recall, 'train/f1-score': train_f,
                           'train/tp': train_tp, 'train/fp': train_fp, 'train/tn': train_tn, 'train/fn': train_fn})
            
                (test_loss, test_acc, test_precision, test_recall, test_f,
                 test_tp, test_fp, test_tn, test_fn, y_true, y_score, test_auc) = test(network, test_loader,
                                                            optimizer, criterion,
                                                            dataset_sizes)
                wandb.log({'test/epoch_loss': test_loss, 'test/epoch_acc': test_acc,
                       'test/precision': test_precision, 'test/recall': test_recall, 'test/f1-score': test_f,
                       'test/tp': test_tp, 'test/fp': test_fp, 'test/tn': test_tn, 'test/fn': test_fn,
                       'test/y_true': y_true, 'test/y_score': y_score, 'test/auc': test_auc})
                if test_auc > best_test_auc:
                    best_y_true = y_true
                    best_y_score = y_score
                    best_test_auc = test_auc
            
            # Get tpr and fpr
            fpr, tpr, thresh = metrics.roc_curve(best_y_true, best_y_score)
            ax.plot(fpr,
                    tpr,
                    label=r"Fold %d (AUC = %0.2f)" % (fold, best_test_auc),
                    lw=1,
                    alpha=0.5)
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(best_test_auc)

            network = network.to(torch.device('cpu'))
            network.eval() 
            # Save model as artifact
            model_pt = wandb.Artifact(
                "trained-model-pt", type="model",
                description="Best Epoch per Fold Pytorch"
            )
            torch.save(network.state_dict(), "resnet-fold-{}.pt".format(fold))
            model_pt.add_file("resnet-fold-{}.pt".format(fold))
            #wandb.save("resnet-fold-{}.pt".format(fold))
            wandb.log_artifact(model_pt)

            # Export model to onnx
            model_onnx = wandb.Artifact(
                "trained-model-onnx", type="model",
                description="Best Epoch per Fold ONNX"
            )
            # Let's create a dummy input tensor  
            dummy_input = torch.randn(1, 3, 224, 224, requires_grad=True)  

            # Export the model   
            torch.onnx.export(network,
                              dummy_input,
                              "resnet-fold-{}.onnx".format(fold),
                              export_params=True,
                              opset_version=11,
                              do_constant_folding=True,
                              input_names = ['input'],
                              output_names = ['output']
            ) 
            model_onnx.add_file("resnet-fold-{}.onnx".format(fold))
            wandb.log_artifact(model_onnx)
            #wandb.save("resnet-fold-{}.onnx".format(fold))

        ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        ax.plot(
            mean_fpr,
            mean_tpr,
            #color="b",
            label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f" % (mean_auc, std_auc),
            lw=1,
            alpha=1,
        )

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        ax.fill_between(
            mean_fpr,
            tprs_lower,
            tprs_upper,
            color="grey",
            alpha=0.4,
            label=r"$\pm$ 1 std. dev.",
        )

        ax.set(
            xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            xlabel="False Positive Rate",
            ylabel="True Positive Rate",
            title=f"Mean ROC curve with variability\n(Positive label `{dataset.classes[1]}')",
        )
        ax.axis("square")
        ax.legend(loc="lower right")
        fig.tight_layout()

        fig.savefig('classifier-hyp-folds.pdf', format='pdf', bbox_inches='tight')
        artifact_plot = wandb.Artifact(name="ROC", type="metric")
        artifact_plot.add_file(
            local_path='classifier-hyp-folds.pdf'
        )
        wandb.log_artifact(artifact_plot)            

In [12]:
sweep_config = {
    'method': 'random'
}

metric = {
    'name': 'test/f1-score',
    'goal': 'maximize'   
}

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['sgd']
    },
}

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 25},
    'batch_size': {
        'value': 64},
    'learning_rate': {
        'value': 0.01},
    'step_size': {
        'value': 5},
    'k_splits': {
        'value': 10},
})

In [13]:
sweep_id = wandb.sweep(sweep_config, project="classifier-optimized")

Create sweep with ID: bq0rvyfn
Sweep URL: https://wandb.ai/flower-classification/classifier-optimized/sweeps/bq0rvyfn


In [ ]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: qxhbaz0l with config:
wandb: 	batch_size: 64
wandb: 	epochs: 25
wandb: 	k_splits: 10
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
wandb: 	step_size: 5


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Dataset targets:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 206MB/s]


  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Fold 9
Dataset sizes:  {'train': 814, 'test': 90}


  0%|          | 0/25 [00:00<?, ?it/s]

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Fold 10
Dataset sizes:  {'train': 814, 'test': 90}


  0%|          | 0/25 [00:00<?, ?it/s]